## Pricing Asian Style Options
In this cript we show how to use classes in *QMCPy* for Monte Carlo option pricing of options with Asian style payoffs and European exercise.
 
- The payoff depends on the whole asset price path, not only on the terminal asset price.
- The option is only exercised at expiry, unlike American options, which can be exercised at any time before expiry.


In [1]:
# Import necessary packages
import qmcpy as qp
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import time

### European Options

In [29]:
initPrice = 120; # initial stock price
interest = 0.02 # risk-free interest rate
volatility = 0.5 # volatility
callput = 'call' # call options
strike = 130 # strike price
tfinal = 1/4 # mature time
d = 4 # number of observations
absTol = 0.05 # absolute tolerance of a nickel
relTol = 0 # zero relative tolerance
sampleSize = 2**4 # numbbeer of smaple size


In [30]:
eo = qp.EuropeanOption(qp.DigitalNetB2(dimension=d,seed=7), volatility= volatility,start_price= initPrice,
                       strike_price=strike, interest_rate = interest,t_final=1/4,call_put=callput)
x = eo.discrete_distrib.gen_samples(sampleSize)
y = eo.f(x)
print("The exact price of this European Option is ",f"{eo.get_exact_value():.4f}")
print("After generate ", sampleSize,"digital net instances, the estimation of the fair price is",y.mean())

The exact price of this European Option is  8.2779
After generate  16 digital net instances, the estimation of the fair price is nan


/Users/yding/Documents/GitHub/QMCSoftware/qmcpy/true_measure/gaussian.py:75


In [32]:
eo

EuropeanOption (Integrand Object)
    volatility      2^(-1)
    call_put        call
    start_price     120
    strike_price    130
    interest_rate   0.020

In [33]:
ac = qp.AsianOption(qp.DigitalNetB2(dimension=d,seed=7), volatility= volatility,start_price= initPrice,
                       strike_price=strike, interest_rate = interest,t_final=1/4,call_put=callput)
ac
x = ac.discrete_distrib.gen_samples(sampleSize)
y = ac.f(x)
print("After generate ", sampleSize,"digital net instances, the estimation of the fair price is",y.mean())

After generate  16 digital net instances, the estimation of the fair price is nan


/Users/yding/Documents/GitHub/QMCSoftware/qmcpy/true_measure/gaussian.py:75


In [34]:
x

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [35]:
ac = qp.AsianOption(qp.DigitalNetB2(4,seed=7))
x = ac.discrete_distrib.gen_samples(2**12)
y = ac.f(x)
y.mean()

/Users/yding/Documents/GitHub/QMCSoftware/qmcpy/true_measure/gaussian.py:75


nan